In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [19]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection (1) (1)/Brain_Tumor_Detection/Train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection (1) (1)/Brain_Tumor_Detection/Train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection (1) (1)/Brain_Tumor_Detection/pred',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2440 images belonging to 2 classes.
Found 609 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [21]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [22]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
77/77 [==============================] - 372s 5s/step - loss: 0.4946 - accuracy: 0.7807 - val_loss: 0.2803 - val_accuracy: 0.8916
Epoch 2/5
77/77 [==============================] - 315s 4s/step - loss: 0.2006 - accuracy: 0.9270 - val_loss: 0.1270 - val_accuracy: 0.9688
Epoch 3/5
77/77 [==============================] - 312s 4s/step - loss: 0.1036 - accuracy: 0.9652 - val_loss: 0.1278 - val_accuracy: 0.9491
Epoch 4/5
77/77 [==============================] - 291s 4s/step - loss: 0.0429 - accuracy: 0.9877 - val_loss: 0.0345 - val_accuracy: 0.9918
Epoch 5/5
77/77 [==============================] - 318s 4s/step - loss: 0.0147 - accuracy: 0.9955 - val_loss: 0.0336 - val_accuracy: 0.9852


In [23]:
model.save("Model.h5","label.txt")


In [ ]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)



In [24]:
image = Image.open("/content/drive/MyDrive/Brain_Tumor_Detection (1) (1)/Brain_Tumor_Detection/pred/pred16.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array


In [25]:
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)


1/1 [==============================] - 0s 147ms/step
Result:  You have Brain Tumor

Accuracy:  0.9999999
